# Training

# Inference

In [11]:
import joblib
from fraudetect.config import load_args_from_json
from fraudetect.preprocessing import FraudFeatureEngineer, FeatureEncoding
from fraudetect.dataset import MyDatamodule, load_data
from fraudetect.config import Arguments
import pandas as pd
from datetime import datetime, date
import os

In [37]:
run = joblib.load(
    "../tools/runs-optuna/small-models_2025-04-16_11-37_best-run.joblib"
)
# results, transform_pipe, datamodule, selector = run

In [38]:
results=run[0]
results

OptunaSearchCV(cv=TimeSeriesSplit(gap=5255, max_train_size=None, n_splits=3, test_size=None),
               estimator=Pipeline(steps=[('data_processor',
                                          Pipeline(steps=[('fraudfeatureengineer',
                                                           FraudFeatureEngineer(session_gap_minutes=180,
                                                                                uid_cols=(None,),
                                                                                windows_size_in_days=(1,
                                                                                                      7,
                                                                                                      30))),
                                                          ('columndropper',
                                                           ColumnDropper(cols_to_drop=['CurrencyCode',
                                                                                       'CountryCode',
                                                                                       'Subscri...
                                    'model__min_samples_leaf': CategoricalDistribution(choices=(1, 2)),
                                    'model__min_samples_split': CategoricalDistribution(choices=(2, 3, 4)),
                                    'model__random_state': CategoricalDistribution(choices=(None,)),
                                    'model__splitter': CategoricalDistribution(choices=('best',))},
               random_state=41, scoring='f1',
               study=<optuna.study.study.Study object at 0x7f3d07c82570>,
               timeout=180)

In [39]:
clf = results.best_estimator_
clf

Pipeline(steps=[('data_processor',
                 Pipeline(steps=[('fraudfeatureengineer',
                                  FraudFeatureEngineer(session_gap_minutes=180,
                                                       uid_cols=(None,),
                                                       windows_size_in_days=(1,
                                                                             7,
                                                                             30))),
                                 ('columndropper',
                                  ColumnDropper(cols_to_drop=['CurrencyCode',
                                                              'CountryCode',
                                                              'SubscriptionId',
                                                              'BatchId',
                                                              'CUSTOMER_ID',
                                                              'AccountId',
                                                              'TRANSACTION_ID',
                                                              'TX_DATETIME',
                                                              'TX_TIME_DAYS'])),
                                 ('featureencoding',
                                  FeatureEncoding(imputer_n_neighbors=9))])),
                ('model',
                 DecisionTreeClassifier(class_weight='balanced', max_depth=15,
                                        max_features='sqrt'))])

In [40]:
args, cfg = load_args_from_json(
    "../tools/runs-optuna/small-models_2025-04-16_11-37.json"
)

In [41]:
args

Arguments(data_path='../data/training.csv', study_name='small-models_2025-04-16_11-37', work_dir='runs-optuna', run_name='debug', delta_train=50, delta_delay=7, delta_test=20, random_state=41, windows_size_in_days=[1, 7, 30], sampler_names=None, sampler_cfgs=None, model_names=['decisionTree', 'logisticReg', 'svc'], session_gap_minutes=180, onehot_threshold=9, pyod_detectors=['abod', 'cblof', 'hbos', 'iforest', 'knn', 'loda', 'mcd'], disable_pyod_outliers=False, disable_samplers=True, do_pca=True, do_poly_expansion=False, do_feature_selection=True, cv_n_iter=200, cv_gap=5255, cv_method='optuna', n_splits=3, n_jobs=4, scoring='f1', cat_encoding_method='binary', cat_encoding_base_n=4, cat_encoding_hash_method='md5', cat_encoding_hash_n_components=7, add_imputer=False, concat_features=[None], optuna_n_trials=20, cols_to_drop=['CurrencyCode', 'CountryCode', 'SubscriptionId', 'BatchId', 'CUSTOMER_ID', 'AccountId', 'TRANSACTION_ID', 'TX_DATETIME', 'TX_TIME_DAYS'])

In [42]:
raw_data_train = load_data("../data/training.csv")

raw_data_pred = load_data("../data/test.csv")

step: load data
step: load data
There is no column FraudResult in loaded data.


In [43]:
raw_data_train.shape

(95662, 17)

In [44]:
raw_data_pred.shape

(45019, 16)

In [45]:
y_pred = clf.predict(raw_data_pred)

In [46]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.], shape=(45019,))

In [47]:
y_pred.sum(), y_pred.sum()/y_pred.shape[0]

(np.float64(298.0), np.float64(0.006619427352895444))

In [ ]:
results, transform_pipe, col_transformer = run

for k,v in results.items():
    try:
        print(v.best_score_)
    except:
        pass

In [48]:
test_data = pd.read_csv("../data/test.csv")
test_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [49]:
# make submission
submission = pd.read_csv("../data/sample_submission.csv")
submission.head()

,TransactionId,FraudResult
0,TransactionId_50600,NaN
1,TransactionId_95109,NaN
2,TransactionId_47357,NaN
3,TransactionId_28185,NaN
4,TransactionId_22140,NaN


In [50]:
## Great same...
(test_data['TransactionId'] == submission['TransactionId']).sum()

np.int64(45019)

In [51]:
submission['FraudResult'] = y_pred

submission['FraudResult'] = submission['FraudResult'].astype('int')
submission.head()

,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0


In [52]:
submission['FraudResult'].sum()

np.int64(298)

In [53]:
current_time = datetime.now().strftime("%H-%M")
filename = f"submission_{str(date.today())}_{current_time}.csv"
filename = os.path.join("../data",filename)

submission.to_csv(filename,index=False)